# Escola de Dados - Python para inovação cívica
## Módulo 1: Noções básicas de estatística descritiva e pré-processamento

- [Aula 8](#Aula-8:-Introdução-a-estatística)
- [Aula 9](#Aula-9:-Medidas-de-tendência-central-e-dispersão)
- [Aula 10](#Aula-10:-Outliers-e-valores-faltantes)
- [Aula 12](#Aula-12:-Operações-básicas-com-python)
- [Aula 13](#Aula-13:-Métodos-`filter`-e-`sort_values`)
- [Aula 14](#Aula-14:-Operações-com-dados-e-método-`apply`)
- [Aula 15](#Aula-15:-Operações-com-dados-e-método-`groupby`)


### Aula 8: Introdução a estatística

Os dados que usaremos nesse notebook pode ser encontrado em: [TSE. Candidaturas em 2020](https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1).

Trabalharemos com os dados referentes as candidaturas nos estados de Pernambuco e Rio Grande do Norte.


In [1]:
import pandas as pd

In [2]:
dados_pe_rn = ['../conjunto_dados/consulta_cand_2020_PE.csv', '../conjunto_dados/consulta_cand_2020_RN.csv']

In [3]:
lista_df = []

for dados_estado in dados_pe_rn:
    df_estado = pd.read_csv(dados_estado, sep=';', encoding='latin_1')
    lista_df.append(df_estado)

FileNotFoundError: [Errno 2] No such file or directory: '../conjunto_dados/consulta_cand_2020_PE.csv'

In [ ]:
df_completo = pd.concat(lista_df, axis=0, ignore_index=True)

É possível criar um dataframe apenas com os dados de candidaturas para as Câmeras de Vereados dos estados.

In [ ]:
df_vereadores = df_completo[df_completo['CD_CARGO'] == 13].copy()

In [ ]:
df_vereadores.columns

O método `pd.describe` traz um quadro com algumas medidas da estatística descritiva, como as medidas de tendência central, medidas de dispersão e quartis, excluindo os valores NaN (not a number).

In [ ]:
df_vereadores.describe()

Para gerar relatórios interativos em HTML a partir de um dataframe do pandas usaremos a biblioteca Pandas Profiling, com ele para cada coluna será apresentado estatístico correspondente ao tipo de dado.

In [ ]:
import numpy as np
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(df_vereadores, title="Introdução - Pandas Profiling Report")

In [ ]:
profile.to_file(output_file="relatorio_eleicoes_vereadores_2020_pe_rn.html")

## Em PE e RN, houveram quantas candidaturas para a prefeitura?

### Aula 9: medidas de tendência central e dispersão

As medidas de tendência central servem para determinar o valor central de uma distribuição. Vamos explorar mais um pouco nosso dataframe e descobrir qual a moda (valor que mais se repete) da colina 'SG_PARTIDO'

In [ ]:
df_vereadores['SG_PARTIDO'].mode()

A média aritmética da coluna 'NR_IDADE_DATA_POSSE' é:

In [ ]:
df_vereadores['NR_IDADE_DATA_POSSE'].mean()

Continuando a explorar a idade das pessoas candidatas na data da posse, queremos saber qual é o valor que ocupa a posição central do conjunto de dados, após a ordenação dos valores (mediana).


In [ ]:
df_vereadores['NR_IDADE_DATA_POSSE'].median()

Qual a variância do dado 'NR_IDADE_DATA_POSSE'?

In [ ]:
df_vereadores['NR_IDADE_DATA_POSSE'].var()

Qual o desvio padrão do dado 'NR_IDADE_DATA_POSSE'?

In [ ]:
df_vereadores['NR_IDADE_DATA_POSSE'].std()

## Qual a média, moda e mediana do 'VR_DESPESA_MAX_CAMPANHA'?

### Aula 10: Outliers e valores faltantes


De acordo com o dicionário de dados do conjunto de dados do TSE, os valores faltantes estão preenchidos com #NULO#, no entanto o Pandas não reconhece essa palavra como um missing. Faremos a substituição da string "#NULO#" pelo valor `None` nas colunas com valores categóricos.

In [ ]:
colunas_categoricas = ['NM_TIPO_ELEICAO', 'DS_ELEICAO', 'TP_ABRANGENCIA', 'SG_UF', 'NM_UE', 'DS_CARGO', 
                       'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'NM_SOCIAL_CANDIDATO', 'NM_EMAIL', 
                       'DS_SITUACAO_CANDIDATURA', 'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'SG_PARTIDO', 
                       'NM_PARTIDO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO', 'DS_NACIONALIDADE', 
                       'SG_UF_NASCIMENTO', 'NM_MUNICIPIO_NASCIMENTO', 'DS_GENERO', 'DS_GRAU_INSTRUCAO', 
                       'DS_ESTADO_CIVIL', 'DS_COR_RACA', 'DS_OCUPACAO', 'VR_DESPESA_MAX_CAMPANHA', 
                       'DS_SIT_TOT_TURNO', 'ST_REELEICAO', 'ST_DECLARAR_BENS', 'DS_SITUACAO_CANDIDATO_PLEITO', 
                       'DS_SITUACAO_CANDIDATO_URNA', 'ST_CANDIDATO_INSERIDO_URNA'
                      ]


In [ ]:
df_vereadores[colunas_categoricas] = df_vereadores.loc[:, colunas_categoricas].replace({'#NULO#': None})

In [ ]:
df_vereadores.info()

Geraremos um novo relatório usando o Pandas Profiling para visualizar as mudanças.

In [ ]:
profile = ProfileReport(df_vereadores, title="Limpeza dos dados - Pandas Profiling Report")
profile.to_file(output_file="relatorio_eleicoes_vereadores_2020_pe_rn_limpo.html")

### Aula 12: Operações básicas com dados

Costumamos construir um subdataframe apenas com as colunas que usaremos na nossa análise. Existem várias formas de filtrar as linhas e colunas. Vamos aprender como fazer isso separadamente: 

Filtrar as 26 colunas pertinentes para nosso estudo.

In [ ]:
df_vereadores_pe_rn_reduzido = df_vereadores[
    ['ANO_ELEICAO','TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_CARGO', 'DS_CARGO',
     'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO','NM_SOCIAL_CANDIDATO', 'NR_PARTIDO',
     'SG_PARTIDO', 'NR_IDADE_DATA_POSSE','CD_GENERO', 'DS_GENERO', 'CD_COR_RACA', 'DS_COR_RACA',
     'VR_DESPESA_MAX_CAMPANHA', 'CD_SIT_TOT_TURNO','DS_SIT_TOT_TURNO', 'ST_REELEICAO', 'CD_SITUACAO_CANDIDATURA',
     'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND', 'DS_DETALHE_SITUACAO_CAND']
].copy()

In [ ]:
df_vereadores_pe_rn_reduzido.shape

Agora selecionaremos apenas as pessoas que tem sua candidaturas "apto", "deferido" e "deferido com recurso".

In [ ]:
df_vereadores_pe_rn_reduzido_aptas = df_vereadores_pe_rn_reduzido[
    df_vereadores_pe_rn_reduzido['CD_SITUACAO_CANDIDATURA'] == 12
]

In [ ]:
df_vereadores_pe_rn_reduzido_aptas_deferidas = df_vereadores_pe_rn_reduzido_aptas[
    (df_vereadores_pe_rn_reduzido_aptas['DS_DETALHE_SITUACAO_CAND'] == 'DEFERIDO') | 
    (df_vereadores_pe_rn_reduzido_aptas['DS_DETALHE_SITUACAO_CAND'] == 'DEFERIDO COM RECURSO')
]

In [ ]:
df_vereadores_pe_rn_reduzido_aptas_deferidas.shape

Agora, aprenderemos a filtrar usando a função `pd.loc`:

In [ ]:
colunas_desejadas = ['ANO_ELEICAO','TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_CARGO', 'DS_CARGO',
     'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO','NM_SOCIAL_CANDIDATO', 'NR_PARTIDO',
     'SG_PARTIDO', 'NR_IDADE_DATA_POSSE','CD_GENERO', 'DS_GENERO', 'CD_COR_RACA', 'DS_COR_RACA',
     'VR_DESPESA_MAX_CAMPANHA', 'CD_SIT_TOT_TURNO','DS_SIT_TOT_TURNO', 'ST_REELEICAO', 'CD_SITUACAO_CANDIDATURA',
     'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND', 'DS_DETALHE_SITUACAO_CAND']

In [ ]:
filtro_linhas = (df_vereadores['CD_SITUACAO_CANDIDATURA'] == 12) & (
    (df_vereadores['DS_DETALHE_SITUACAO_CAND'] == 'DEFERIDO') | 
    (df_vereadores['DS_DETALHE_SITUACAO_CAND'] == 'DEFERIDO COM RECURSO')
)

In [ ]:
df_vereadores_pe_rn_reduzido_aptas_deferidas_2 = df_vereadores.loc[filtro_linhas, colunas_desejadas]

In [ ]:
df_vereadores_pe_rn_reduzido_aptas_deferidas_2.shape

In [ ]:
df_vereadores_pe_rn = df_vereadores_pe_rn_reduzido_aptas_deferidas_2.copy()

## Sua vez, crie um dataframe com apenas os nomes e partidos das pessoas candidatas que se autodeclararam como 'PARDA', 'PRETA' e 'INDÍGENA'

### Aula 13: Métodos `filter` e `sort_values`

Vamos filtrar todas as colunas que possuam as letras "DIDATO" no seu título:

In [ ]:
df_vereadores_pe_rn_nomes = df_vereadores_pe_rn.filter(like='DIDATO', axis=1)
df_vereadores_pe_rn_nomes.head()

Para ordenar as linhas a partir das colunas 'NM_SOCIAL_CANDIDATO' e 'NM_CANDIDATO, usamos o método df.sort_values:

In [ ]:
df_vereadores_pe_rn_nomes.sort_values(by=['NM_SOCIAL_CANDIDATO', 'NM_CANDIDATO'], 
                                      ascending=True, 
                                      na_position='last', 
                                      ignore_index=False
)

## Selecione apenas as colunas cujo o título possua "DS" e organize em ordem descrescente. Qual o gênero das candidaturas das 3 primeiras linhas?

### Aula 14: Operações com dados e método `apply`

Podemos criar uma nova coluna a partir de operações sobre outras colunas. No exemplo abaixo vamos descobrir o gasto médio diário declarado por cada candidatura.

In [ ]:
total_dias_campanha = 45
df_vereadores_pe_rn['GASTO_DIARIO'] = df_vereadores_pe_rn['VR_DESPESA_MAX_CAMPANHA'] / total_dias_campanha

In [ ]:
df_vereadores_pe_rn.head()

Explorando mais um pouco esses dados, trabalharemos apenas com os dados das candidaturas de Pernambuco e classificaremos os gastos médios diários de acordo com os conceitos de quartil e valores discrepantes que vimos nos vídeos sobre estatística:

In [ ]:
df_vereadores_pe = df_vereadores_pe_rn[df_vereadores_pe_rn['SG_UF'] == 'PE'].copy()

In [ ]:
primeiro_quartil = df_vereadores_pe['GASTO_DIARIO'].quantile(0.25)
terceiro_quartil = df_vereadores_pe['GASTO_DIARIO'].quantile(0.75)

In [ ]:
intervalo_interquartil = terceiro_quartil - primeiro_quartil

In [ ]:
limite_inferior = primeiro_quartil - (intervalo_interquartil * 1.5)
limite_superior = terceiro_quartil + (intervalo_interquartil * 1.5)

In [ ]:
def classificacao_gasto(row):
    if row < limite_inferior:
        return 'muito abaixo da média'
    elif row >= limite_inferior and row < primeiro_quartil:
        return 'abaixo da média'
    elif row >= primeiro_quartil and row <= terceiro_quartil:
        return 'na média'
    elif row > terceiro_quartil and row <= limite_superior:
        return 'acima da média'
    else:
        return 'muito acima da média'

In [ ]:
df_vereadores_pe['CLASSIFICACAO_GASTO_DIARIO'] = df_vereadores_pe['GASTO_DIARIO'].apply(classificacao_gasto)

In [ ]:
df_vereadores_pe.head()

Para facilitar a visualização faremos um gráfico de barras usando a biblioteca matplotlib:

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# guarda na variável "classificacao" uma lista com os rótulos usados para classificar os gastos
classificacao = df_vereadores_pe['CLASSIFICACAO_GASTO_DIARIO'].unique().tolist()

In [ ]:
#guarda na variável "total_por_classificacao" uma lista a quantidade de ocorrência de cada rótulo
total_por_classificacao = df_vereadores_pe['CLASSIFICACAO_GASTO_DIARIO'].value_counts().to_list()

In [ ]:
plt.bar(classificacao, total_por_classificacao)
plt.title('Gastos diários declarados por candidatura')
plt.ylabel('Quantitativo de municípios por classificação')

plt.show()

### Aula 15: Operações com dados e método `groupby`

Usando o método groupby podemos agrupar dados segundo informações de uma coluna e fazer operações com o agrupamento.

As perguntas que queremos responder são: quantas mulheres são candidatas? Qual a distribuição de pessoas por grupo racial? Quantidade de mulheres por grupo racial? E qual partido tem mais mulheres em cada grupo racial?


In [ ]:
colunas_desejadas = ['SG_PARTIDO', 'DS_GENERO', 'DS_COR_RACA']
filtro = df_vereadores_pe_rn['SG_UF'] == 'PE'

In [ ]:
df_vereadores_pe_sexo_raca = df_vereadores_pe_rn.loc[filtro, colunas_desejadas]

In [ ]:
df_vereadores_pe_sexo_raca.head()

Agrupando por gênero, quantas candidaturas temos em cada um?

In [ ]:
df_vereadores_pe_sexo_raca.groupby(['DS_GENERO']).count()

E por raça?

In [ ]:
df_vereadores_pe_sexo_raca.groupby(['DS_COR_RACA']).count()

Da candidaturas do gênero feminino, quantas temos em cada grupo racial?

In [ ]:
df_vereadores_pe_sexo_raca[df_vereadores_pe_sexo_raca['DS_GENERO'] == 'FEMININO'].groupby(
    ['DS_GENERO', 'DS_COR_RACA']).count()

Entre as candidaturas do gênero feminino, quais os partidos com maior número de representação em cada grupo racial?

In [ ]:
df_vereadores_pe_sexo_raca[df_vereadores_pe_sexo_raca['DS_GENERO'] == 'FEMININO'].groupby(
    ['DS_GENERO', 'DS_COR_RACA']).max()

## Agora é a sua vez, entre as candidaturas do sexo masculino, quais os partidos com o menor número de representantes em cada grupo racial?